In [5]:
import kagglehub
import os
import shutil

dataset_ref = "ckay16/accident-detection-from-cctv-footage"

datasets_path = '../../datasets/'

path = kagglehub.dataset_download(handle = dataset_ref)

os.makedirs(datasets_path, exist_ok=True)

dataset_name = os.path.basename(path)
shutil.move(path, os.path.join(datasets_path, dataset_name))

print(path)

C:\Users\harne\.cache\kagglehub\datasets\ckay16\accident-detection-from-cctv-footage\versions\1
